## This notebook converts point data loaded as a geodataframe into a multiband raster

In [1]:
import geopandas as gpd
import rasterio as rio

In [ ]:
gdf = gpd.read_file(r"C:\Users\coach\myfiles\miscellenous\hayley\datasets\all_response_groups_bii_8km\subset.shp")
gdf.explore()

In [7]:
gdf.columns
column_prefixes = ['B', 'HA', 'HR', 'LC', 'PR', 'SC', 'SMB', 'SMI', 'SMR', 'LH']
columns = [col for col in gdf.columns if any(col.startswith(prefix) for prefix in column_prefixes)]


In [9]:
len(columns)

113

In [ ]:
import geopandas as gpd
import numpy as np
from osgeo import gdal, ogr, osr
from scipy.spatial import cKDTree

def calculate_resolution(gdf):
    """
    Calculate the approximate resolution (pixel size) based on the spacing between points.
    
    Parameters:
    - gdf: Input GeoDataFrame with point geometries.
    
    Returns:
    - resolution: Average distance between neighboring points.
    """
    # Extract coordinates of points
    coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

    # Build a KDTree for efficient neighbor lookup
    tree = cKDTree(coords)

    # Query the nearest neighbor distances for all points
    distances, _ = tree.query(coords, k=2)  # k=2 because the first nearest is the point itself
    avg_distance = np.mean(distances[:, 1])  # Take the second nearest (index 1)
    
    return avg_distance

shp = r"C:\Users\coach\myfiles\miscellenous\hayley\datasets\all_response_groups_bii_8km\subset.shp"
calculate_resolution(gpd.read_file(shp))

0.07836747496292432

In [10]:
import geopandas as gpd
from osgeo import gdal, ogr, osr
import fiona
from tqdm.auto import tqdm

def geodf_to_multiband_tif(gdf_path, attributes, output_raster, resolution, no_data_value, expand_by=2, tile_size=1024):
    """
    Converts a large GeoDataFrame with points and attributes into a multiband raster TIFF efficiently and ensures alignment.
    
    Parameters:
    - gdf_path: Path to the shapefile.
    - attributes: List of attribute names to rasterize into separate bands.
    - output_raster: Path for the output raster (multiband TIFF).
    - resolution: Pixel size (same for x and y).
    - no_data_value: No data value to set in the raster.
    - expand_by: Factor to slightly expand the raster extent to ensure all points are covered.
    - tile_size: Size of the raster tile/block for efficient processing.
    """
    
    # Open the input vector using Fiona (better for large datasets)
    with fiona.open(gdf_path, 'r') as source:
        xmin, ymin, xmax, ymax = source.bounds
    
    # Adjust bounds to ensure the rightmost and bottommost points are included (expand extent slightly)
    xmin -= expand_by * resolution
    ymin -= expand_by * resolution
    xmax += expand_by * resolution
    ymax += expand_by * resolution

    # Create the output raster with LZW compression
    driver = gdal.GetDriverByName("GTiff")
    raster_ds = driver.Create(output_raster, int((xmax - xmin) / resolution),
                              int((ymax - ymin) / resolution), len(attributes), 
                              gdal.GDT_Float32, options=['COMPRESS=LZW', 'TILED=YES', f'BLOCKXSIZE={tile_size}', f'BLOCKYSIZE={tile_size}'])
    
    # Set the geotransform (tie point and pixel size)
    raster_ds.SetGeoTransform((xmin, resolution, 0, ymax, 0, -resolution))

    # Set the projection to WGS 84 (EPSG:4326)
    output_crs = osr.SpatialReference()
    output_crs.ImportFromEPSG(4326)
    raster_ds.SetProjection(output_crs.ExportToWkt())

    # Open the shapefile for streaming processing
    mem_driver = gdal.GetDriverByName('Memory')
    
    # Rasterize each attribute into separate bands using tiling
    for i, attribute in tqdm(enumerate(attributes), total=len(attributes)):
        mem_ds = mem_driver.Create('', 0, 0, 0, gdal.GDT_Unknown)  # Temporary memory dataset
        gdal.VectorTranslate(mem_ds, gdf_path, format='ESRI Shapefile', options=['-select', attribute])  # Only select relevant attribute
        layer = mem_ds.GetLayer()

        # Rasterize in blocks for each attribute
        for block_x in range(0, raster_ds.RasterXSize, tile_size):
            for block_y in range(0, raster_ds.RasterYSize, tile_size):
                # Read block-by-block to avoid loading everything into memory
                gdal.RasterizeLayer(raster_ds, [i + 1], layer, options=[f"ATTRIBUTE={attribute}", f"NODATA={no_data_value}"])
                
        band = raster_ds.GetRasterBand(i + 1)
        band.SetNoDataValue(no_data_value)
        band.SetDescription(attribute)  # Set the band description to the attribute name

    # Flush cache and close datasets
    raster_ds.FlushCache()
    raster_ds = None

# Example usage:
shp = r"C:\Users\coach\myfiles\miscellenous\hayley\datasets\all_response_groups_bii_8km\subset.shp"
column_prefixes = ['B', 'HA', 'HR', 'LC', 'PR', 'SC', 'SMB', 'SMI', 'SMR', 'LH']
attributes = [col for col in gpd.read_file(shp).columns if any(col.startswith(prefix) for prefix in column_prefixes)]  # List of attributes to rasterize
output_raster = r"C:\Users\coach\myfiles\miscellenous\hayley\datasets\output_multiband.tif"
resolution = 0.07836747496292432  # Same resolution for both x and y
no_data_value = -999

geodf_to_multiband_tif(shp, attributes, output_raster, resolution, no_data_value, 0.5)


  0%|          | 0/113 [00:00<?, ?it/s]

In [12]:
# run for entire dataset
# Use the subset to get the column names of interest- faster thatn reading in the entire point shp
shp = r"C:\Users\coach\myfiles\miscellenous\hayley\datasets\all_response_groups_bii_8km\subset.shp"
column_prefixes = ['B', 'HA', 'HR', 'LC', 'PR', 'SC', 'SMB', 'SMI', 'SMR', 'LH']
attributes = [col for col in gpd.read_file(shp).columns if any(col.startswith(prefix) for prefix in column_prefixes)]
shp = r"C:\Users\coach\myfiles\miscellenous\hayley\datasets\all_response_groups_bii_8km\all_response_groups_bii_8km.shp"
output_raster = r"C:\Users\coach\myfiles\miscellenous\hayley\datasets\all_response_groups_bii_8km.tif"
resolution = 0.07836747496292432  # Same resolution for both x and y
no_data_value = -999

geodf_to_multiband_tif(shp, attributes, output_raster, resolution, no_data_value, 1.5)

  0%|          | 0/113 [00:00<?, ?it/s]